In [ ]:
## vscode 

# python: 인터프리터 선택 > conda env 동일하게 선택

In [ ]:
# !pip install torch torchvision numpy matplotlib

In [ ]:
# imports
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler

from torch.utils.tensorboard import SummaryWriter

import torchvision
from torchvision import transforms, datasets

import numpy as np
import matplotlib.pyplot as plt

In [ ]:
if torch.cuda.is_available():
    device = torch.device('cuda')
# elif torch.backends.mps.is_available():
#     device = torch.device('mps')
else:
    device = torch.device('cpu')

print("Using PyTorch version:", torch.__version__,' Device:', device)

In [ ]:
# transforms
transform = transforms.Compose(
    [transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))])

# datasets
train_set = datasets.FashionMNIST('./data',
    download=True,
    train=True,
    transform=transform)
test_set = datasets.FashionMNIST('./data',
    download=True,
    train=False,
    transform=transform)

# dataloaders
BATCH_SIZE = 32

train_loader = torch.utils.data.DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=BATCH_SIZE, shuffle=False)

In [ ]:
writer = SummaryWriter('runs/fashion_mnist_experiment')

In [ ]:
# vs code 없는 사람: 
# command: tensorboard --logdir=runs
# browser: https://localhost:6006

In [ ]:
def matplotlib_imshow(img, one_channel=False):
    if one_channel:
        img = img.mean(dim=0)
    img = img / 2 + 0.5
    npimg = img.numpy()
    if one_channel:
        plt.imshow(npimg, cmap="Greys")
    else:
        plt.imshow(np.transpose(npimg, (1, 2, 0)))

dataiter = iter(train_loader)
images, labels = next(dataiter)

img_grid = torchvision.utils.make_grid(images)
matplotlib_imshow(img_grid, one_channel=True)

writer.add_image('four_fashion_mnist_images', img_grid)

In [ ]:
# classes
classes = ('T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
        'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle Boot')

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(28*28, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 10)
        
    def forward(self,x):
        x = x.view(-1, 28*28)
        x = self.fc1(x)
        x = F.sigmoid(x)
        x = self.fc2(x)
        x = F.sigmoid(x)
        x = self.fc3(x)
        x = F.log_softmax(x, dim=1)
        return x

In [ ]:
# Model
model = Net().to(device)

# Optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Loss function
criterion = nn.CrossEntropyLoss()

# scheduler
scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

print(model)

In [ ]:
writer.add_graph(model, images)

In [ ]:
def visualize_samples_tensorboard(loader, classes, samples_per_class=5):
    class_dict = {i: [] for i in range(len(classes))}
    
    for img, label in loader:
        for i in range(len(img)):
            if len(class_dict[label[i].item()]) < samples_per_class:
                class_dict[label[i].item()].append(img[i])
        if all(len(v) == samples_per_class for v in class_dict.values()):
            break
    
    for label, images in class_dict.items():
        grid = torchvision.utils.make_grid(images, nrow=samples_per_class, normalize=True)
        writer.add_image(f'{classes[label]} samples', grid, 0)

visualize_samples_tensorboard(train_loader, classes)

In [ ]:
def select_n_random(data, labels, n=100):
    '''
    데이터셋에서 n개의 임의의 데이터포인트(datapoint)와 그에 해당하는 라벨을 선택합니다
    '''
    assert len(data) == len(labels)

    perm = torch.randperm(len(data))
    return data[perm][:n], labels[perm][:n]

images, labels = select_n_random(train_set.data, train_set.targets)

class_labels = [classes[lab] for lab in labels]

features = images.view(-1, 28 * 28)
writer.add_embedding(features,
                    metadata=class_labels,
                    label_img=images.unsqueeze(1))

def images_to_probs(net, images):
    '''
    학습된 신경망과 이미지 목록으로부터 예측 결과 및 확률을 생성합니다
    '''
    output = net(images)
    _, preds_tensor = torch.max(output, 1)
    preds = np.squeeze(preds_tensor.numpy())
    return preds, [F.softmax(el, dim=0)[i].item() for i, el in zip(preds, output)]


def plot_classes_preds(net, images, labels):
    '''
    학습된 신경망과 배치로부터 가져온 이미지 / 라벨을 사용하여 matplotlib
    Figure를 생성합니다. 이는 신경망의 예측 결과 / 확률과 함께 정답을 보여주며,
    예측 결과가 맞았는지 여부에 따라 색을 다르게 표시합니다. "images_to_probs"
    함수를 사용합니다.
    '''
    preds, probs = images_to_probs(net, images)
    fig = plt.figure(figsize=(12, 48))
    for idx in np.arange(4):
        ax = fig.add_subplot(1, 4, idx+1, xticks=[], yticks=[])
        matplotlib_imshow(images[idx], one_channel=True)
        ax.set_title("{0}, {1:.1f}%\n(label: {2})".format(
            classes[preds[idx]],
            probs[idx] * 100.0,
            classes[labels[idx]]),
                    color=("green" if preds[idx]==labels[idx].item() else "red"))
    return fig

In [ ]:
# 훈련 함수
def train(model, train_loader, optimizer, scheduler, criterion, epoch, device, writer=None):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        # parameter initialization
        optimizer.zero_grad()
        
        # forward pass
        output = model(data)
        
        # calculate loss
        loss = criterion(output, target)
        
        # backward pass
        loss.backward()
        
        # optimize
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(output.data, 1)
        total += target.size(0)
        correct += (predicted == target).sum().item()

    epoch_loss = running_loss / len(train_loader.dataset)
    epoch_accuracy = 100. * correct / total

    print(f'Train Epoch: {epoch} Loss: {epoch_loss:.4f} Accuracy: {epoch_accuracy:.2f}%')
    
    scheduler.step()
    writer.add_scalar('Loss/train', epoch_loss, epoch)
    writer.add_scalar('Accuracy/train', epoch_accuracy, epoch)
    
    writer.add_figure('predictions vs. actuals', plot_classes_preds(model, data, target), global_step=epoch)

# 테스트 함수
def test(model, test_loader, criterion, epoch, device, writer=None):
    model.eval()
    test_loss = 0
    correct = 0
    total = 0

    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            loss = criterion(output, target)
            test_loss += loss.item()
            _, predicted = torch.max(output.data, 1)
            total += target.size(0)
            correct += (predicted == target).sum().item()

    test_loss /= len(test_loader.dataset)
    test_accuracy = 100. * correct / total

    print(f'Test Epoch: {epoch} Loss: {test_loss:.4f} Accuracy: {test_accuracy:.2f}%')
    writer.add_scalar('Loss/test', test_loss, epoch)
    writer.add_scalar('Accuracy/test', test_accuracy, epoch)

# 훈련 및 테스트 실행
epochs = 50

for epoch in range(1, epochs + 1):
    train(model, train_loader, optimizer, scheduler, criterion, epoch, device, writer)
    test(model, test_loader, criterion, epoch, device, writer)

In [ ]:
# writer.close()